In [1]:
from pathlib import Path
import sys

root_dir = Path().resolve().parent.parent.parent.as_posix()
if root_dir not in sys.path:
    sys.path.append(root_dir)
del root_dir

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import efficientnet_v2
import tensorflow as tf

from utils import config, datasets, models

In [2]:
MODEL_NAME = 'efficientnetv2b0-3_layer_dropout'
EPOCHS = 100

In [3]:
train_dataset, validation_dataset, test_dataset = datasets.load_dataset()

Found 6400 files belonging to 4 classes.


In [4]:
data_augmentation = keras.Sequential([
    layers.RandomFlip(),
    layers.RandomRotation(factor=0.25),
    layers.RandomContrast(factor=0.2),
    layers.RandomBrightness(factor=0.2),
    models.RandomSaturation(2, 16),
    models.RandomHue(0.4),
    layers.GaussianNoise(stddev=0.2),
])

In [5]:
top = keras.Sequential([
    layers.Dense(2048, activation=keras.activations.relu),
    layers.Dropout(0.2),
    layers.Dense(2048, activation=keras.activations.relu),
    layers.Dropout(0.2),
    layers.Dense(2048, activation=keras.activations.relu),
    layers.Dropout(0.2),
    layers.Dense(4, activation=keras.activations.softmax)
])

In [6]:
model = models.create_model(
    model_name=MODEL_NAME,
    preprocessing_layers=data_augmentation,
    base_model=efficientnet_v2.EfficientNetV2B0,
    top_layers=top,
    pooling='avg',
)

In [7]:
history = models.fit_model(
    model,
    train_data=train_dataset,
    validation_data=validation_dataset,
    epochs=EPOCHS
)

Epoch 1/100
160/160 [==============================] - 67s 326ms/step - loss: 1.7464 - sparse_categorical_accuracy: 0.4809 - val_loss: 0.9818 - val_sparse_categorical_accuracy: 0.5188
Epoch 2/100
160/160 [==============================] - 50s 311ms/step - loss: 1.0081 - sparse_categorical_accuracy: 0.5129 - val_loss: 0.9540 - val_sparse_categorical_accuracy: 0.5453
Epoch 3/100
160/160 [==============================] - 50s 312ms/step - loss: 0.9827 - sparse_categorical_accuracy: 0.5170 - val_loss: 0.9748 - val_sparse_categorical_accuracy: 0.5453
Epoch 4/100
160/160 [==============================] - 50s 311ms/step - loss: 0.9949 - sparse_categorical_accuracy: 0.5201 - val_loss: 0.9862 - val_sparse_categorical_accuracy: 0.5625
Epoch 5/100
160/160 [==============================] - 50s 312ms/step - loss: 0.9732 - sparse_categorical_accuracy: 0.5180 - val_loss: 0.9365 - val_sparse_categorical_accuracy: 0.5750
Epoch 6/100
160/160 [==============================] - 50s 312ms/step - loss: 0.

In [8]:
model.summary()

Model: "efficientnetv2b0-3_layer_dropout"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (None, 128, 128, 3)       0         
                                                                 
 efficientnetv2-b0 (Function  (None, 1280)             5919312   
 al)                                                             
                                                                 
 sequential_1 (Sequential)   (None, 4)                 11024388  
                                                                 
Total params: 16,943,700
Trainable params: 11,024,388
Non-trainable params: 5,919,312
_________________________________________________________________
